In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, get_extreme_values, create_directory_structure, save_table, save_report, pd, assign_quartile
# settings
region_column_name = 'Region'
table_name = 'security_headers_by_region'
report_name = 'security_headers_by_region'
category = 'security_headers'
column_name_to_results_global = 'Global #'
create_directory_structure()

source_df = load_data('security_headers_checker')

In [2]:
# sanity dataset
headers = [
            'strict-transport-security',
            'x-frame-options',
            'x-content-type-options',
            'content-security-policy',
            'x-permitted-cross-domain-policies',
            'referrer-policy',
            'clear-site-data',
            'cross-origin-embedder-policy',
            'cross-origin-opener-policy',
            'cross-origin-resource-policy',
            'cache-control'
        ]

# Records that are null suffered an error during data collection and are therefore treated as not having security headers.
for header in headers:
    source_df.loc[source_df[header].isna(), header] = False

source_df.info()

In [3]:
# Analysis of HEIs with at least one OWASP Recommended Security Headers by region (Pub/Pvt)

# settings
column_to_sort = 'Without Security Headers %'
sort_ascending = True
config = [
    {'table_name': 'security_headers_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'security_headers_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    # Column creation with distribution of records without any security headers by region
    only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
    criteria = f'category == "{hei_type}" & ({only_headers_false})'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers', criteria=criteria, columns_to_display=columns_to_display)
    # Column creation with distribution of records with at least one security header by region
    all_true_filter = ' & '.join([f"`{h}` == True" for h in headers])
    at_least_one_filter = ' | '.join([f"`{h}` == True" for h in headers])
    composite_filter = f"({at_least_one_filter}) & ~({all_true_filter})"
    criteria = f'category == "{hei_type}" & ({composite_filter})'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header', criteria=criteria, columns_to_display=columns_to_display)
    # Column creation with distribution of records with all security header by region
    criteria = f'category == "{hei_type}" & ({all_true_filter})'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers', criteria=criteria, columns_to_display=columns_to_display)
    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)



df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')


#add column with sum of best columns
df_combined['Rank'] = df_combined['With All Security Headers (pub)'] + df_combined['With All Security Headers (pvt)']
df_combined['Rank2'] = df_combined['With At Least One Security Header (pub)'] + df_combined['With At Least One Security Header (pvt)']


#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=['Rank', 'Rank2'], ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=['Rank', 'Rank2'], inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='security_headers_by_region_combined')

,Region,Global #,Without Security Headers (Public) #,Without Security Headers (Public) %,Without Security Headers (Private) #,Without Security Headers (Private) %,With At Least One Security Header (Public) #,With At Least One Security Header (Public) %,With At Least One Security Header (Private) #,With At Least One Security Header (Private) %,With All Security Headers (Public) #,With All Security Headers (Public) %,With All Security Headers (Private) #,With All Security Headers (Private) %
0,California,2,0,0.000000,0,0.000000,0,0.0,2,100.000000,0,0.0,0,0.0
1,Florida,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.000000,0,0.0,0,0.0
3,Illinois,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
4,Michigan,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
5,Minnesota,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
6,New York,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.0
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.000000,0,0.0,0,0.0
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
9,Total,11,1,9.090909,3,27.272727,0,0.0,7,63.636364,0,0.0,0,0.0


In [5]:
# specific security headers by region (Pub/Pvt)

# settings
column_to_sort = 'Without Security Headers %'
sort_ascending = True
config = [
    {'table_name': 'security_headers_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'security_headers_by_region_private', 'hei_type': 'Private'}
]
for config_item in config:
    hei_type = config_item['hei_type']
    for header in headers:
        table_name = f'{header.lower()}_by_region_{hei_type.lower()}'
        report_name = f'{header.lower()}_by_region_{hei_type.lower()}'
        columns_to_display = [region_column_name, column_name_to_results_global]
        analysis_df = get_records_by_region(source_df, hei_type=hei_type)

        # Column creation with distribution of records without any security headers by region
        only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
        criteria = f'category == "{hei_type}" & ({only_headers_false})'
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers', criteria=criteria, columns_to_display=columns_to_display)
        # Column creation with distribution of records with a specific security header by region
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header}", criteria=f'category == "{hei_type}"  & (`{header}` == True)', columns_to_display=columns_to_display)

        # Column creation with distribution of records without a specific security header by region
        true_at_least_one_header_except_current = ' | '.join([f"`{h}` == True" for h in headers if h != header])
        criteria = f"(`{header}` == False) & ({true_at_least_one_header_except_current})"
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header}", criteria=f'category == "{hei_type}" & {criteria}', columns_to_display=columns_to_display)
        display(analysis_df)

        # Finalize dataframe
        analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
        display(analysis_df)
        # save to csv
        save_table(analysis_df, category=category, table_name=table_name)


,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,Without Security Headers (Private) #,Without Security Headers (Private) %,With strict-transport-security (Public) #,With strict-transport-security (Public) %,With strict-transport-security (Private) #,With strict-transport-security (Private) %,Without strict-transport-security (Public) #,Without strict-transport-security (Public) %,Without strict-transport-security (Private) #,Without strict-transport-security (Private) %
region,,,,,,,,,,,,,,,,,,,,,
California,2,2,2,2,2,2,2,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,100.0
Colorado,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Florida,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Georgia,1,1,1,1,1,1,1,1,1,1,...,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Illinois,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0
Michigan,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Minnesota,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0
New York,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Texas,2,2,2,2,2,2,2,2,2,2,...,2.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Region,Global #,Without Security Headers (Public) #,Without Security Headers (Public) %,Without Security Headers (Private) #,Without Security Headers (Private) %,With strict-transport-security (Public) #,With strict-transport-security (Public) %,With strict-transport-security (Private) #,With strict-transport-security (Private) %,Without strict-transport-security (Public) #,Without strict-transport-security (Public) %,Without strict-transport-security (Private) #,Without strict-transport-security (Private) %
0,California,2,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,2,100.000000
1,Florida,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
3,Illinois,1,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,1,100.000000
4,Michigan,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
5,Minnesota,1,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,1,100.000000
6,New York,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
9,Total,11,1,9.090909,3,27.272727,0,0.0,3,27.272727,0,0.0,4,36.363636



    \begin{figure}[htbp]
        \centering
        \includegraphics[width=0.48\textwidth]{charts/strict-transport-security_by_region.pdf}
        \caption{Distribution of strict-transport-security header usage by region.}\label{fig:strict-transport-security}
    \end{figure}
    

The data presented in Figure~\ref{fig:strict-transport-security} provides an overview of the use of strict-transport-security security headers at \glspl{hei} in \countryName. According to the data, 0.00\% of the public institutions analyzed have not implemented strict-transport-security security header on their websites, while 36.36\% of the private institutions analyzed also lack strict-transport-security security header.

On a positive note, 0.00\% of the public institutions analyzed have implemented strict-transport-security security headers on their websites, while 27.27\% of the private institutions analyzed also implemented that security header.

In terms of regional differences, private institutions 

,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,Without Security Headers (Private) #,Without Security Headers (Private) %,With x-frame-options (Public) #,With x-frame-options (Public) %,With x-frame-options (Private) #,With x-frame-options (Private) %,Without x-frame-options (Public) #,Without x-frame-options (Public) %,Without x-frame-options (Private) #,Without x-frame-options (Private) %
region,,,,,,,,,,,,,,,,,,,,,
California,2,2,2,2,2,2,2,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,100.0
Colorado,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Florida,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Georgia,1,1,1,1,1,1,1,1,1,1,...,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Illinois,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0
Michigan,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Minnesota,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0
New York,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0
Texas,2,2,2,2,2,2,2,2,2,2,...,2.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Region,Global #,Without Security Headers (Public) #,Without Security Headers (Public) %,Without Security Headers (Private) #,Without Security Headers (Private) %,With x-frame-options (Public) #,With x-frame-options (Public) %,With x-frame-options (Private) #,With x-frame-options (Private) %,Without x-frame-options (Public) #,Without x-frame-options (Public) %,Without x-frame-options (Private) #,Without x-frame-options (Private) %
0,California,2,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,2,100.000000
1,Florida,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
3,Illinois,1,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,1,100.000000
4,Michigan,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
5,Minnesota,1,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,1,100.000000
6,New York,1,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,1,100.000000
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
9,Total,11,1,9.090909,3,27.272727,0,0.0,2,18.181818,0,0.0,5,45.454545


...

,Region,Global #,Without Security Headers (Public) #,Without Security Headers (Public) %,Without Security Headers (Private) #,Without Security Headers (Private) %,With cross-origin-resource-policy (Public) #,With cross-origin-resource-policy (Public) %,With cross-origin-resource-policy (Private) #,With cross-origin-resource-policy (Private) %,Without cross-origin-resource-policy (Public) #,Without cross-origin-resource-policy (Public) %,Without cross-origin-resource-policy (Private) #,Without cross-origin-resource-policy (Private) %
0,California,2,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,2,100.000000
1,Florida,1,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,1,100.000000
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.000000
3,Illinois,1,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,1,100.000000
4,Michigan,1,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,1,100.000000
5,Minnesota,1,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,1,100.000000
6,New York,1,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,1,100.000000
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.0,0,0.0,0,0.000000
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000
9,Total,11,1,9.090909,3,27.272727,0,0.0,0,0.0,0,0.0,7,63.636364



    \begin{figure}[htbp]
        \centering
        \includegraphics[width=0.48\textwidth]{charts/cross-origin-resource-policy_by_region.pdf}
        \caption{Distribution of cross-origin-resource-policy header usage by region.}\label{fig:cross-origin-resource-policy}
    \end{figure}
    

The data presented in Figure~\ref{fig:cross-origin-resource-policy} provides an overview of the use of cross-origin-resource-policy security headers at \glspl{hei} in \countryName. According to the data, 0.00\% of the public institutions analyzed have not implemented cross-origin-resource-policy security header on their websites, while 63.64\% of the private institutions analyzed also lack cross-origin-resource-policy security header.

On a positive note, 0.00\% of the public institutions analyzed have implemented cross-origin-resource-policy security headers on their websites, while 0.00\% of the private institutions analyzed also implemented that security header.

In terms of regional differences

,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,Without Security Headers (Private) #,Without Security Headers (Private) %,With cache-control (Public) #,With cache-control (Public) %,With cache-control (Private) #,With cache-control (Private) %,Without cache-control (Public) #,Without cache-control (Public) %,Without cache-control (Private) #,Without cache-control (Private) %
region,,,,,,,,,,,,,,,,,,,,,
California,2,2,2,2,2,2,2,2,2,2,...,0.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,0.0
Colorado,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Florida,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Georgia,1,1,1,1,1,1,1,1,1,1,...,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Illinois,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Michigan,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
Minnesota,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0
New York,1,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,100.0
Texas,2,2,2,2,2,2,2,2,2,2,...,2.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Region,Global #,Without Security Headers (Public) #,Without Security Headers (Public) %,Without Security Headers (Private) #,Without Security Headers (Private) %,With cache-control (Public) #,With cache-control (Public) %,With cache-control (Private) #,With cache-control (Private) %,Without cache-control (Public) #,Without cache-control (Public) %,Without cache-control (Private) #,Without cache-control (Private) %
0,California,2,0,0.000000,0,0.000000,0,0.0,2,100.000000,0,0.0,0,0.000000
1,Florida,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
3,Illinois,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
4,Michigan,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
5,Minnesota,1,0,0.000000,0,0.000000,0,0.0,1,100.000000,0,0.0,0,0.000000
6,New York,1,0,0.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,1,100.000000
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.000000
9,Total,11,1,9.090909,3,27.272727,0,0.0,6,54.545455,0,0.0,1,9.090909



    \begin{figure}[htbp]
        \centering
        \includegraphics[width=0.48\textwidth]{charts/cache-control_by_region.pdf}
        \caption{Distribution of cache-control header usage by region.}\label{fig:cache-control}
    \end{figure}
    

The data presented in Figure~\ref{fig:cache-control} provides an overview of the use of cache-control security headers at \glspl{hei} in \countryName. According to the data, 0.00\% of the public institutions analyzed have not implemented cache-control security header on their websites, while 9.09\% of the private institutions analyzed also lack cache-control security header.

On a positive note, 0.00\% of the public institutions analyzed have implemented cache-control security headers on their websites, while 54.55\% of the private institutions analyzed also implemented that security header.

In terms of regional differences, private institutions in California (100.00\%), Florida (100.00\%), and Illinois (100.00\%) have a higher usage of cach